In [89]:
import numpy as np
import scipy.special as spe
from math import sqrt
from keras.utils import to_categorical

Regression Dataset - Diabetes

In [2]:
import sklearn.datasets as sd
import sklearn.model_selection as sm
import sklearn.preprocessing as sp

std = sp.MinMaxScaler(feature_range=(0,1))

dataset = sd.load_diabetes()
X = std.fit_transform(dataset["data"])
T = dataset["target"].reshape(-1,1)

X1, X2, T1, T2 = sm.train_test_split(X, T, test_size=0.2, random_state=42)

Classification Dataset - MNIST

In [113]:
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Convert to 1D
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Normalize X to between -1 and 1
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(x_train)
x_train = scaler.transform(x_train)
scaler.fit(x_test)
x_test = scaler.transform(x_test)

# Convert the Target(y) to One Hot Encoded
y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)
y_train = to_categorical(y_train, 10).astype(int)
y_test = to_categorical(y_test, 10).astype(int)

x_val, x_train, y_val, y_train = train_test_split(x_train, y_train, test_size=0.83333)

x_train.shape

(50000, 784)

dRVFL Functions

In [103]:
def generateNodes(lmbda, n_feature, n_nodes):
    W = lmbda * (2 * np.random.rand(n_feature, n_nodes) - 1)
    b = lmbda * (2 * np.random.rand(1, n_nodes) - 1)
    return W, b
    
def activationFunc(X, W, b):
    return spe.expit(X @ W + b)

def combineH(Hi, I):
    return np.concatenate([Hi, I], axis=1)

def combineD(H, X):
    return np.concatenate([np.ones_like(X[:,0:1]), H, X], axis=1)

def calcBeta(D, T, n_nodes, n_sample):
    return np.linalg.pinv(D) @ T

def predict(Beta, X, D, T):
    Y = D @ Beta
    resErr = Y - T
    return resErr, Y

def calcRMSE(resErr, n_sample):
    return sqrt(np.sum(np.sum(resErr ** 2, axis=0) / n_sample, axis=0))

def oneHot(T):
    T = sp.LabelEncoder().fit_transform(T)
    return to_categorical(T).astype(int)

def calcAcc(Y, T):
    


Training Phase - Regression

In [91]:
# Define parameters
n_nodes_per_layer = [200, 100, 80]
lmbdas = 20
n_sample = X1.shape[0]

I = X1.copy()

Wi = list()
bi = list()
Hi = list()

for n_nodes in n_nodes_per_layer:
    
    n_feature = I.shape[1]
    W, b = generateNodes(lmbdas, n_feature, n_nodes)

    I = activationFunc(I, W, b)
    
    if len(Hi) == 0:
        Hi = I
    else:
        Hi = combineH(Hi, I)

    Wi.append(W)
    bi.append(b)
    
D = combineD(Hi, X1)

Beta = calcBeta(D, T1, sum(n_nodes_per_layer), n_sample)

resErr, Y = predict(Beta, X1, D, T1)

score = calcRMSE(resErr, n_sample)

print(f"RMSE Score: {score:.4f}")

RMSE Score: 9.9119


Training Phase - Classification

In [115]:
# Define Parameters
n_nodes_per_layer = [1887, 140, 50]
lmbdas = 200
n_sample = x_train.shape[0]

I = x_train.copy()

Wi = list()
bi = list()
Hi = list()

for n_nodes in n_nodes_per_layer:
    
    n_feature = I.shape[1]
    W, b = generateNodes(lmbdas, n_feature, n_nodes)

    I = activationFunc(I, W, b)
    
    if len(Hi) == 0:
        Hi = I
    else:
        Hi = combineH(Hi, I)

    Wi.append(W)
    bi.append(b)
    
D = combineD(Hi, x_train)

Beta = calcBeta(D, y_train, sum(n_nodes_per_layer), n_sample)

resErr, Y = predict(Beta, x_train, D, y_train)

score = calcRMSE(resErr, n_sample)

print(f"RMSE Score: {score:.4f}")

RMSE Score: 0.4510
